<a href="https://colab.research.google.com/github/anaperez10/Analiticalll-Finanzas/blob/main/Analitica_3_Seguros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Conexión con colab
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import plotly.graph_objs as go #Graficos
import plotly.express as px
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt ### gráficos
from plotly.subplots import make_subplots

In [ ]:
expustos= pd.read_table('/BD_Expuestos.txt')
socio =pd.read_table('/BD_Sociodemograficas.txt', encoding='latin-1')
siniestros =pd.read_table('/DB_Siniestros.txt',encoding='latin-1')

Base de datos de expuestos

In [ ]:
expustos.head(5)

,Asegurado_Id,Poliza_Asegurado_Id,FECHA_INICIO,FECHA_CANCELACION,FECHA_FIN
0,16286173,125696117,2018/01/01,2018/04/10,2018/04/10
1,11885152,141320760,2018/01/01,NaN,2019/12/31
2,763957,162532851,2018/01/01,NaN,2019/12/31
3,29082204,154620473,2018/01/01,2018/04/19,2018/04/19
4,4567640,93633862,2018/01/01,NaN,2019/12/31


In [ ]:
expustos.columns

Index(['Asegurado_Id', 'Poliza_Asegurado_Id', 'FECHA_INICIO',
       'FECHA_CANCELACION', 'FECHA_FIN'],
      dtype='object')

In [ ]:
expustos.columns= expustos.columns.str.lower()

In [ ]:
expustos.dtypes

asegurado_id            int64
poliza_asegurado_id     int64
fecha_inicio           object
fecha_cancelacion      object
fecha_fin              object
dtype: object

In [ ]:
expustos.shape

(300900, 5)

In [ ]:
expustos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300900 entries, 0 to 300899
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   asegurado_id         300900 non-null  int64 
 1   poliza_asegurado_id  300900 non-null  int64 
 2   fecha_inicio         300900 non-null  object
 3   fecha_cancelacion    151963 non-null  object
 4   fecha_fin            300900 non-null  object
dtypes: int64(2), object(3)
memory usage: 11.5+ MB


In [ ]:
expustos['fecha_inicio']=pd.to_datetime(expustos['fecha_inicio'])
expustos['fecha_cancelacion']=pd.to_datetime(expustos['fecha_cancelacion'])
expustos['fecha_fin']=pd.to_datetime(expustos['fecha_fin'])

In [ ]:
expustos.dtypes

asegurado_id                    int64
poliza_asegurado_id             int64
fecha_inicio           datetime64[ns]
fecha_cancelacion      datetime64[ns]
fecha_fin              datetime64[ns]
dtype: object

In [ ]:
expustos=expustos.fillna(0)

In [ ]:
def categorizar(fila):
    fecha_cancelacion = fila.fecha_cancelacion
    if fecha_cancelacion  == 0:
      return 0
    else:
      return 1


In [ ]:
expustos["Cancelacion"] = expustos.apply(categorizar, axis=1)

In [ ]:
expustos

,asegurado_id,poliza_asegurado_id,fecha_inicio,fecha_cancelacion,fecha_fin,Cancelacion
0,16286173,125696117,2018-01-01,2018-04-10 00:00:00,2018-04-10,1
1,11885152,141320760,2018-01-01,0,2019-12-31,0
2,763957,162532851,2018-01-01,0,2019-12-31,0
3,29082204,154620473,2018-01-01,2018-04-19 00:00:00,2018-04-19,1
4,4567640,93633862,2018-01-01,0,2019-12-31,0
...,...,...,...,...,...,...
300895,28476983,53646742,2018-01-01,0,2019-12-31,0
300896,8479396,89963102,2018-01-01,2018-03-08 00:00:00,2018-03-08,1
300897,6598270,165960557,2018-02-28,0,2019-12-31,0
300898,1000907,105811540,2018-01-01,0,2019-12-31,0


base de datos de siniestros

In [ ]:
siniestros.head(5)

,Mes_Pago,Asegurado_Id,Reclamacion,Diagnostico_Codigo,Diagnostico_Desc,Eventos,Valor_Pagado
0,201803,4834968,EXAMENES DE DIAGNOSTICO,Z011,EXAMEN DE OÍDOS Y DE LA AUDICIÓN,1,111600.00
1,201909,14915125,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,1,50468.00
2,201807,42959949,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,1,610452.00
3,201803,30540131,CONSULTA EXTERNA,9,DIAGNÓSTICO PENDIENTE,1,42160.00
4,201905,21442420,LABORATORIO CLINICO,9,DIAGNÓSTICO PENDIENTE,18,617336.48


In [ ]:
siniestros.columns

Index(['Mes_Pago', 'Asegurado_Id', 'Reclamacion', 'Diagnostico_Codigo',
       'Diagnostico_Desc', 'Eventos', 'Valor_Pagado'],
      dtype='object')

In [ ]:
siniestros.dtypes

Mes_Pago                int64
Asegurado_Id            int64
Reclamacion            object
Diagnostico_Codigo     object
Diagnostico_Desc       object
Eventos                 int64
Valor_Pagado          float64
dtype: object

In [ ]:
siniestros.shape

(3308480, 7)

In [ ]:
siniestros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3308480 entries, 0 to 3308479
Data columns (total 7 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Mes_Pago            int64  
 1   Asegurado_Id        int64  
 2   Reclamacion         object 
 3   Diagnostico_Codigo  object 
 4   Diagnostico_Desc    object 
 5   Eventos             int64  
 6   Valor_Pagado        float64
dtypes: float64(1), int64(3), object(3)
memory usage: 176.7+ MB


In [ ]:
siniestros.isnull().sum()

Mes_Pago              0
Asegurado_Id          0
Reclamacion           0
Diagnostico_Codigo    0
Diagnostico_Desc      0
Eventos               0
Valor_Pagado          0
dtype: int64

In [ ]:
siniesros=siniestros.dropna()

In [ ]:
#Imprimir categorias de las variables
lista = ['Reclamacion','Diagnostico_Codigo','Diagnostico_Desc']
for i in lista:
    print(siniestros[i].value_counts())
    print('------------------------------------')

CONSULTA EXTERNA                                                   1255793
EXAMENES DE DIAGNOSTICO                                             694238
LABORATORIO CLINICO                                                 520214
CONSULTA DE URGENCIAS                                               365872
CIRUGIA AMBULATORIA POR ENFERMEDAD                                  106397
FISIOTERAPIA ILIMITADA                                               98370
TRATAMIENTOS ESPECIFICOS                                             49708
TRATAMIENTO MEDICO AMBULATORIO POR ACCIDENTES                        39360
CONSULTAS PRENATALES                                                 33685
TRATAMIENTO MEDICO HOSPITALARIO POR ENFERMEDAD                       28504
TRATAMIENTO QUIRURGICO HOSPITALARIO POR ENFERMEDAD                   20484
CIRUGIA AMBULATORIA POR ACCIDENTE                                    17594
TRATAMIENTO AMBULATORIO POR CANCER                                   14949
INFILTRACIONES           

Base de datos sociodemograficas.

In [ ]:
socio.head(5)

,Asegurado_Id,Sexo_Cd,FechaNacimiento,Regional,CANCER,EPOC,DIABETES,HIPERTENSION,ENF_CARDIOVASCULAR
0,18686469,F,1984/01/17,REGIONAL ANTIOQUIA,0,0,0,0,0
1,16087775,M,1984/03/12,REGIONAL NORTE,0,0,0,0,0
2,35757761,F,1993/12/19,REGIONAL ANTIOQUIA,0,0,0,0,0
3,48691460,M,2000/11/14,REGIONAL ANTIOQUIA,0,0,0,0,0
4,36074099,F,2015/08/06,REGIONAL CENTRO,0,0,0,0,0


In [ ]:
socio.columns

Index(['Asegurado_Id', 'Sexo_Cd', 'FechaNacimiento', 'Regional', 'CANCER',
       'EPOC', 'DIABETES', 'HIPERTENSION', 'ENF_CARDIOVASCULAR'],
      dtype='object')

In [ ]:
socio.dtypes

Asegurado_Id           int64
Sexo_Cd               object
FechaNacimiento       object
Regional              object
CANCER                 int64
EPOC                   int64
DIABETES               int64
HIPERTENSION           int64
ENF_CARDIOVASCULAR     int64
dtype: object

In [ ]:
socio['FechaNacimiento']=pd.to_datetime(socio['FechaNacimiento'])

In [ ]:
socio.shape

(267312, 9)

In [ ]:
socio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267312 entries, 0 to 267311
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Asegurado_Id        267312 non-null  int64         
 1   Sexo_Cd             267312 non-null  object        
 2   FechaNacimiento     267312 non-null  datetime64[ns]
 3   Regional            267311 non-null  object        
 4   CANCER              267312 non-null  int64         
 5   EPOC                267312 non-null  int64         
 6   DIABETES            267312 non-null  int64         
 7   HIPERTENSION        267312 non-null  int64         
 8   ENF_CARDIOVASCULAR  267312 non-null  int64         
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 18.4+ MB


In [ ]:
#Imprimir categorias de las variables
lista = ['Sexo_Cd','Regional','CANCER','EPOC','DIABETES','HIPERTENSION','ENF_CARDIOVASCULAR']
for i in lista:
    print(socio[i].value_counts())
    print('------------------------------------')

F     145430
M     121877
-1         5
Name: Sexo_Cd, dtype: int64
------------------------------------
REGIONAL CENTRO          127149
REGIONAL ANTIOQUIA        49722
REGIONAL OCCIDENTE        46180
REGIONAL NORTE            34845
REGIONAL EJE CAFETERO      9342
Sin Información              73
Name: Regional, dtype: int64
------------------------------------
0    262929
1      4383
Name: CANCER, dtype: int64
------------------------------------
0    266055
1      1257
Name: EPOC, dtype: int64
------------------------------------
0    263774
1      3538
Name: DIABETES, dtype: int64
------------------------------------
0    250566
1     16746
Name: HIPERTENSION, dtype: int64
------------------------------------
0    266387
1       925
Name: ENF_CARDIOVASCULAR, dtype: int64
------------------------------------


In [ ]:
#Pasar los nulos de usuarios a sin información
socio['Sexo_Cd'] = socio['Sexo_Cd'].replace('-1','F')

In [ ]:
socio=socio.dropna()

In [ ]:
socio['edad']=2019-socio.FechaNacimiento.dt.year

In [ ]:
socio['edad'].value_counts()

 35     6748
 38     6745
 39     6735
 37     6587
 34     6526
        ... 
-2         3
 102       3
-3         2
 103       2
 105       1
Name: edad, Length: 109, dtype: int64

In [ ]:
socio.loc[socio[socio.edad<0].index,['edad']]=0

In [ ]:
def categorizar1(fila):
    edad = fila.edad
    if edad  < 6:
      return 'Primera infancia'
    elif edad < 11:
      return 'Infancia'
    elif edad < 18:
      return 'Adolescencia'
    elif edad < 28:
      return 'Jovenes'
    elif edad <59:
      return'Adultez'
    elif edad >60:
      return 'Persona Mayor'


In [ ]:
socio["curso de vida"] = socio.apply(categorizar1, axis=1)

Union de las bases de datos.

In [ ]:
dfcx = pd.merge(socio,siniestros, on = 'Asegurado_Id',how = 'inner')
dfcx.head(3)

,Asegurado_Id,Sexo_Cd,FechaNacimiento,Regional,CANCER,EPOC,DIABETES,HIPERTENSION,ENF_CARDIOVASCULAR,edad,curso de vida,Mes_Pago,Reclamacion,Diagnostico_Codigo,Diagnostico_Desc,Eventos,Valor_Pagado
0,18686469,F,1984-01-17,REGIONAL ANTIOQUIA,0,0,0,0,0,35,Adultez,201912,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,3,1027462.76
1,18686469,F,1984-01-17,REGIONAL ANTIOQUIA,0,0,0,0,0,35,Adultez,201912,CONSULTA EXTERNA,9,DIAGNÓSTICO PENDIENTE,1,67208.00
2,16087775,M,1984-03-12,REGIONAL NORTE,0,0,0,0,0,35,Adultez,201810,CONSULTA DE URGENCIAS,9,DIAGNÓSTICO PENDIENTE,1,83111.00


In [ ]:
dfcx.isnull().sum()

Asegurado_Id              0
Sexo_Cd                   0
FechaNacimiento           0
Regional                  0
CANCER                    0
EPOC                      0
DIABETES                  0
HIPERTENSION              0
ENF_CARDIOVASCULAR        0
edad                      0
curso de vida         80806
Mes_Pago                  0
Reclamacion               0
Diagnostico_Codigo        0
Diagnostico_Desc          0
Eventos                   0
Valor_Pagado              0
dtype: int64

In [ ]:
expustos1 = expustos.rename(columns={'asegurado_id':'Asegurado_Id'})

In [ ]:
dfcx1 = pd.merge(dfcx,expustos1, on = 'Asegurado_Id',how = 'inner')
dfcx1.head(3)

,Asegurado_Id,Sexo_Cd,FechaNacimiento,Regional,CANCER,EPOC,DIABETES,HIPERTENSION,ENF_CARDIOVASCULAR,edad,...,Reclamacion,Diagnostico_Codigo,Diagnostico_Desc,Eventos,Valor_Pagado,poliza_asegurado_id,fecha_inicio,fecha_cancelacion,fecha_fin,Cancelacion
0,18686469,F,1984-01-17,REGIONAL ANTIOQUIA,0,0,0,0,0,35,...,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,3,1027462.76,202863951,2019-10-31,0,2019-12-31,0
1,18686469,F,1984-01-17,REGIONAL ANTIOQUIA,0,0,0,0,0,35,...,CONSULTA EXTERNA,9,DIAGNÓSTICO PENDIENTE,1,67208.00,202863951,2019-10-31,0,2019-12-31,0
2,16087775,M,1984-03-12,REGIONAL NORTE,0,0,0,0,0,35,...,CONSULTA DE URGENCIAS,9,DIAGNÓSTICO PENDIENTE,1,83111.00,139441379,2018-01-01,2019-12-31 00:00:00,2019-12-31,1


In [ ]:
dfcx1.shape

(3781424, 22)

In [ ]:
dfcx1.isnull().sum()

Asegurado_Id           0
Sexo_Cd                0
FechaNacimiento        0
Regional               0
CANCER                 0
EPOC                   0
DIABETES               0
HIPERTENSION           0
ENF_CARDIOVASCULAR     0
edad                   0
Mes_Pago               0
Reclamacion            0
Diagnostico_Codigo     0
Diagnostico_Desc       0
Eventos                0
Valor_Pagado           0
poliza_asegurado_id    0
fecha_inicio           0
fecha_cancelacion      0
fecha_fin              0
Cancelacion            0
curso de vida          0
dtype: int64

In [ ]:
dfcx1=dfcx1.drop(['curso de vida'], axis=1)

In [ ]:
dfcx1["curso de vida"] = dfcx1.apply(categorizar1, axis=1)

In [ ]:
dfcx1

,Asegurado_Id,Sexo_Cd,FechaNacimiento,Regional,CANCER,EPOC,DIABETES,HIPERTENSION,ENF_CARDIOVASCULAR,edad,...,Diagnostico_Codigo,Diagnostico_Desc,Eventos,Valor_Pagado,poliza_asegurado_id,fecha_inicio,fecha_cancelacion,fecha_fin,Cancelacion,curso de vida
0,18686469,F,1984-01-17,REGIONAL ANTIOQUIA,0,0,0,0,0,35,...,9,DIAGNÓSTICO PENDIENTE,3,1027462.76,202863951,2019-10-31,0,2019-12-31,0,Adultez
1,18686469,F,1984-01-17,REGIONAL ANTIOQUIA,0,0,0,0,0,35,...,9,DIAGNÓSTICO PENDIENTE,1,67208.00,202863951,2019-10-31,0,2019-12-31,0,Adultez
2,16087775,M,1984-03-12,REGIONAL NORTE,0,0,0,0,0,35,...,9,DIAGNÓSTICO PENDIENTE,1,83111.00,139441379,2018-01-01,2019-12-31 00:00:00,2019-12-31,1,Adultez
3,16087775,M,1984-03-12,REGIONAL NORTE,0,0,0,0,0,35,...,9,DIAGNÓSTICO PENDIENTE,1,194166.64,139441379,2018-01-01,2019-12-31 00:00:00,2019-12-31,1,Adultez
4,16087775,M,1984-03-12,REGIONAL NORTE,0,0,0,0,0,35,...,9,DIAGNÓSTICO PENDIENTE,1,77729.40,139441379,2018-01-01,2019-12-31 00:00:00,2019-12-31,1,Adultez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3781419,4419287,F,1950-10-06,REGIONAL NORTE,0,0,0,0,0,69,...,9,DIAGNÓSTICO PENDIENTE,1,66600.40,55104420,2018-01-01,0,2019-12-31,0,Persona Mayor
3781420,4419287,F,1950-10-06,REGIONAL NORTE,0,0,0,0,0,69,...,H269,"CATARATA, NO ESPECIFICADA",1,5727075.16,55104420,2018-01-01,0,2019-12-31,0,Persona Mayor
3781421,4419287,F,1950-10-06,REGIONAL NORTE,0,0,0,0,0,69,...,9,DIAGNÓSTICO PENDIENTE,24,404240.00,55104420,2018-01-01,0,2019-12-31,0,Persona Mayor
3781422,4419287,F,1950-10-06,REGIONAL NORTE,0,0,0,0,0,69,...,9,DIAGNÓSTICO PENDIENTE,3,835885.24,55104420,2018-01-01,0,2019-12-31,0,Persona Mayor


In [ ]:
dfcx1['curso de vida'].value_counts()

Adultez             2097884
Persona Mayor        547524
Primera infancia     386750
Jovenes              301243
Adolescencia         178583
Infancia             176431
Name: curso de vida, dtype: int64

In [ ]:
dfcx1=dfcx1.dropna()

Analisis Exploratorio.

In [ ]:
expustos.columns

Index(['asegurado_id', 'poliza_asegurado_id', 'fecha_inicio',
       'fecha_cancelacion', 'fecha_fin', 'Cancelacion'],
      dtype='object')

In [ ]:
df=pd.Series(expustos.fecha_fin - expustos.fecha_inicio).dt.days.value_counts().reset_index().rename(columns={'index':'Dias',0:'tamaño'})

In [ ]:
#Tiempo en dias en que las personas piden los seguros.
fig=px.bar(df.drop(index=[0,1]),x='Dias', y='tamaño', color_discrete_sequence =  px.colors.sequential.Aggrnyl)
fig.update_layout(font=dict(size=15)) 
fig.show()

In [ ]:
#Distribucion de las personas que cancenlan el seguro antes de terminar
# crear dataset
dic = {0:'No canceló',
       1:'Canceló'}
base = expustos.groupby(['Cancelacion'])[['asegurado_id']].count().sort_values('asegurado_id', ascending = False).reset_index()
base['Cancelacion'] = base['Cancelacion'].replace(dic)
# crear gráfica
fig = px.pie(base, values = 'asegurado_id', names ='Cancelacion',
             title= '<b>Participación de las personas que cancenlan el seguro antes de terminar<b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    template = 'simple_white',
    title_x = 0.5)

fig.show()

In [ ]:
df=pd.Series(expustos.fecha_inicio.dt.month_name().value_counts()).reset_index().rename(columns={'index':'Mes','fecha_inicio':'tamaño'})

In [ ]:
fig=px.bar(df,x='Mes',y='tamaño',color_discrete_sequence =  px.colors.sequential.Aggrnyl )
fig.update_layout(font=dict(size=15))
fig.show()

Exploracion de Siniestros

In [ ]:
siniestros.columns

Index(['Mes_Pago', 'Asegurado_Id', 'Reclamacion', 'Diagnostico_Codigo',
       'Diagnostico_Desc', 'Eventos', 'Valor_Pagado'],
      dtype='object')

In [ ]:
# crear dataset
base = siniestros.groupby(['Diagnostico_Desc'])[['Valor_Pagado']].sum().sort_values('Valor_Pagado', ascending = False).reset_index().head(10)

# crear gráfica
fig = px.bar(base, x = 'Diagnostico_Desc', y='Valor_Pagado',
             title= '<b>Costos por el diagnostico<b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Año',
    yaxis_title = 'Precio de venta',
    template = 'simple_white',
    title_x = 0.5)

fig.show()

In [ ]:
# crear dataset
base = siniestros.groupby(['Reclamacion'])[['Valor_Pagado']].sum().sort_values('Valor_Pagado', ascending = False).reset_index().head(10)

# crear gráfica
fig = px.bar(base, x = 'Reclamacion', y='Valor_Pagado',
             title= '<b>Costos por el tipo de reclamacion<b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Año',
    yaxis_title = 'Precio de venta',
    template = 'simple_white',
    title_x = 0.5)

fig.show()

Exploracion Socio 

In [ ]:
socio.columns

Index(['Asegurado_Id', 'Sexo_Cd', 'FechaNacimiento', 'Regional', 'CANCER',
       'EPOC', 'DIABETES', 'HIPERTENSION', 'ENF_CARDIOVASCULAR', 'edad'],
      dtype='object')

In [ ]:
# crear dataset
base = socio.groupby(['Sexo_Cd'])[['Asegurado_Id']].count().sort_values('Asegurado_Id', ascending = False).reset_index()

# crear gráfica
fig = px.pie(base, values = 'Asegurado_Id', names ='Sexo_Cd',
             title= '<b>Participación del sexo en los seguros de vida b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    template = 'simple_white',
    title_x = 0.5)

fig.show()

In [ ]:
# crear dataset
base = socio.groupby(['curso de vida'])[['Asegurado_Id']].count().sort_values('Asegurado_Id', ascending = False).reset_index()

# crear gráfica
fig = px.pie(base, values = 'Asegurado_Id', names ='curso de vida',
             title= '<b>Participación del curso de vida en los seguros de vida b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    template = 'simple_white',
    title_x = 0.5)

fig.show()

In [ ]:
# crear dataset
base = socio.groupby(['Regional'])[['Asegurado_Id']].count().sort_values('Asegurado_Id', ascending = False).reset_index().head(10)

# crear gráfica
fig = px.bar(base, x = 'Regional', y='Asegurado_Id',
             title= '<b>Cantidad de personas por region<b>',
             color_discrete_sequence=px.colors.qualitative.G10)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Año',
    yaxis_title = 'Precio de venta',
    template = 'simple_white',
    title_x = 0.5)

fig.show()

Analisis exploratorio de base concatenada.

In [ ]:
dfcx1.columns

Index(['Asegurado_Id', 'Sexo_Cd', 'FechaNacimiento', 'Regional', 'CANCER',
       'EPOC', 'DIABETES', 'HIPERTENSION', 'ENF_CARDIOVASCULAR', 'edad',
       'Mes_Pago', 'Reclamacion', 'Diagnostico_Codigo', 'Diagnostico_Desc',
       'Eventos', 'Valor_Pagado', 'poliza_asegurado_id', 'fecha_inicio',
       'fecha_cancelacion', 'fecha_fin', 'Cancelacion', 'curso de vida'],
      dtype='object')

In [ ]:
dfcx1['año'] = dfcx1['fecha_inicio'].dt.year 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# crear dataset

base = dfcx1.groupby(['curso de vida', 'año'])[['Valor_Pagado']].sum().reset_index()
# MSSubClass: Tipo de edificio
# YrSold: Año en que se vende
# SalePrice: Precio de venta

# crear gráfica
fig = px.bar(base, x = 'año', y='Valor_Pagado', color = 'curso de vida', barmode = 'group', title= '<b>costos según el curso de vida por año<b>')

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Año',
    yaxis_title = 'Precio de venta',
    template = 'simple_white',
    title_x = 0.5)

fig.show()

In [ ]:
# crear dataset
base = dfcx1.groupby(['Sexo_Cd', 'año'])[['Valor_Pagado']].sum().reset_index()

# crear gráfica
fig = px.bar(base, x = 'año', y='Valor_Pagado', color = 'Sexo_Cd', barmode = 'group', 
             title= '<b>Precio de venta según el tipo de acceso al edificio<b>',
             color_discrete_sequence=px.colors.qualitative.Antique)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Año',
    yaxis_title = 'Precio de venta',
    template = 'simple_white',
    title_x = 0.5)

fig.show()